In [ ]:
%pylab inline
%load_ext autoreload
%autoreload 2
import numpy as np
import tools  # tools.py, in this directory
from math import tau
from scipy.io import wavfile

pylab.rcParams['figure.figsize'] = 12, 5
𝜏 = tau

In [ ]:
hz = 44100
T = 5             # number of seconds
N = hz * T        # number of samples
n = arange(N)     # samples vector
t = n / hz        # time vector, in seconds
start_hz = 10
stop_hz = 5000

# Creating a test signal

In [ ]:
# if 1:  # linear chirp
#     chirp_rate = (stop_hz - start_hz) / T
#     a = np.sin(𝜏 * (chirp_rate * t * t / 2.0 + start_hz * t))
# else:  # exponential chirp
#     chirp_rate = (stop_hz / start_hz) ** (1 / T)
#     a = np.sin(𝜏 * start_hz * (chirp_rate ** t - 1) / np.log(chirp_rate))
# if 0:  # window
#     window = np.where(n > N/2, N - n, n) / (N/2)
#     a *= window

In [ ]:
# Other waveforms
#a = tools.fuzz(tools.sine_stack(t, 20, 2000, 0.05))
#a = tools.sine_stack(t, 20, 2000, 0.03123)
#print(np.log10(21) - np.log10(20))
a = tools.sine_stack(t, 20, 2000, log10(20.5/20))  # 0.5 Hz resolution at 20 Hz
#a = tools.sine_stack(t, 20, 2000, 0.05)
a = tools.normalize(a)

In [ ]:
plot(t, a, ',')

In [ ]:
f = fft.fft(a) / N
fig, (ax1, ax2) = plt.subplots(2, 1)
ax1.plot(n/T, abs(f), 'o')
ax2.plot(n/T, np.angle(f), ',')
[ax.set_xlim(10, hz/2 - 10) for ax in (ax1, ax2)];
[ax.set_xscale('log') for ax in (ax1, ax2)];
[ax.grid(which='both') for ax in (ax1, ax2)];

In [ ]:
left = 32000 * a
right = 0 * a
stereo = np.array([left, right]).astype(np.int16).T
tools.write('sample.wav', hz, left, right)

# Play sample.wav and measure response

Notes:

```
arecord -f cd -c 1 audio-in.wav -D "HDA Intel PCH"

--duration 10
```

In [ ]:
#output_device = 'plughw:CARD=PCH,DEV=0'
output_device = 'plughw:CARD=USB,DEV=0'
input_device = 'plughw:CARD=USB,DEV=0'

In [ ]:
# ! aplay -D {output_device} sample.wav

In [ ]:
! (sleep 1; aplay -D {output_device} sample.wav) & arecord -f cd -c 2 -D {input_device} --duration={T+2} audio-in.wav
! ls -l audio-in.wav

In [ ]:
hz, channels = wavfile.read('audio-in.wav')
left, right = channels.T
signal = tools.trim_silence(T, hz, right)

In [ ]:
signal[:100]

In [ ]:
print('Maximum amplitude excursion:', max(abs(signal)))
print('Fraction of available 2^32: {:.3f}'.format(max(abs(signal)) / 32768))
plot(signal)

In [ ]:
# Overview plot, to see everything that is going on.
N = len(signal)
f = fft.fft(signal)
fig, (ax1, ax2) = plt.subplots(2, 1)
ax1.plot(hz * arange(N) / N, abs(f), ',')
ax2.plot(hz * arange(N) / N, np.angle(f), ',')
[ax.set_xlim(10, hz/2 - 10) for ax in (ax1, ax2)];

In [ ]:
# Plot narrowed to where we expect signal.
N = len(signal)
f = fft.fft(signal)
fig, ax = plt.subplots()
power_dB = 20 * np.log10(abs(f))
ceiling_dB = max(power_dB) // 5 * 5 + 5
ax.plot(hz * arange(N) / N, power_dB, '.')
#ax2.plot(hz * arange(N) / N, np.angle(f), ',')
ax.set_ylabel('Power (dB)')
ax.set_xlim(start_hz, stop_hz)
ax.set_ylim(ceiling_dB - 50.0, ceiling_dB)
#ax.set_ylim(ceiling_dB - 10.0, ceiling_dB)
ax.set_xscale('log')
ax.grid(which='both')

In [ ]:
x = hz * arange(N) / N
i = 20 * 5
x[i]  # 20 Hz
print(x[i-5:i+6])
print(power_dB[i-5:i+6])

In [ ]:
# Compare!

hz_i, channels = wavfile.read('sample.wav')
left, right = channels.T
input_signal = left

hz_o, channels = wavfile.read('audio-in.wav')
left, right = channels.T
output_signal = right

assert hz_i == hz_o

Ni = len(input_signal)
fi = abs(fft.fft(input_signal))

No = len(output_signal)
fo = abs(fft.fft(output_signal + 2.3e6))

frequency = hz_o * arange(No) / No
print(frequency.shape)
print(fo.shape)
print(arange(Ni).shape)
fi = np.interp(frequency, hz_i * arange(Ni) / Ni, abs(fi))
print(fi.shape)

mask = (frequency >= start_hz) & (frequency <= stop_hz)
frequency = frequency[mask]
fi = fi[mask]
fo = fo[mask]

ratio = fo / fi

fig, [ax1, ax2] = plt.subplots(2, 1)
ax1.set(title='Input and Output spectra, then ratio output ÷ input')
ax1.plot(frequency, fi, '.')
ax1.plot(frequency, fo, '.')
#ax1.plot(frequency, fo + 2.3e6, ',')
ax2.plot(frequency, ratio, '.')
#[ax.set_xlim(start_hz, stop_hz) for ax in [ax1, ax2]];
[ax.set_xscale('log') for ax in [ax1, ax2]];
[ax.grid(which='both') for ax in (ax1, ax2)];

In [ ]:
# NEXT: try adjusting to match amplitude, THEN do FFT. Result SHOULD be same?

# Appendix: should we trim measured signal?

Yes, trimming the measured signal
to remove the quiet on each end
allows a faster FFT
while returning the exact same curve
but with less noise.

In [ ]:
hz, channels = wavfile.read('audio-in.wav')
left, right = channels.T
plot(right)

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1)

for signal in right, tools.trim_silence(right):
    N = len(signal)
    f = fft.fft(signal)
    ax1.plot(hz * arange(N) / N, abs(f), ',')
    ax2.plot(hz * arange(N) / N, np.angle(f), ',')
    [ax.set_xlim(start_hz, stop_hz) for ax in (ax1, ax2)];
    [ax.set_xscale('log') for ax in (ax1, ax2)];

# Appendix: how far apart should we space sine waves?

In [ ]:
k = arange(0.02, 0.09, 0.001)
factors = []
for ki in k:
    a = tools.sine_stack(t, 20, 2000, ki, False)
    factors.append(max(abs(a)))

In [ ]:
plot(k, factors, '.')